In [2]:
"""
@test: 参考書との比較（正しく計算されているか確認）
"""
# ニューラルネットワーク実装読み込み
include("./lib/Neuron.jl")
include("./lib/Functions.jl")

# 数値微分による勾配計算
## 関数(Array{Float64,2})::Float64, 入力値::Array{Float64,2} -> 勾配::Array{Float64,2}
numeric_gradient(f, x::Array{Float64,2})::Array{Float64,2} = begin
    h = 1e-4 # 10^(-4)
    grad = Array{Float64, 2}(undef, size(x, 1), size(x, 2)) # xと同じ次元の行列を生成
    # 各変数ごとの数値微分を行列にまとめる
    for row in 1:size(x, 1), col in 1:size(x, 2)
        # 指定indexの変数に対する中心差分を求める
        org = x[row, col]
        x[row, col] = org + h
        f1 = f(x) # f([..., x[row, col] + h, ...]) -> Float64
        x[row, col] = org - h
        f2 = f(x) # f([..., x[row, col] - h, ...]) -> Float64
        grad[row, col] = (f1 - f2) / 2h # (row, col)番目の変数に対する数値微分
        x[row, col] = org # x[i]の値をもとに戻す
    end
    return grad
end
    
# シンプルなニューラルネットワーク
SimpleNet() = Network(1,
    [
        zeros(1, 3) # 1 x 3 Array{Float64,2} bias_1 [0 0 0]
    ],
    [
        zeros(2, 3) # 2 x 3 Array{Float64,2} weight_1 [0 0 0; 0 0 0]
    ]
)

# 活性関数を使わず、ソフトマックス関数で出力するだけの推論処理
predict(net::Network, x::Array{Float64,2})::Array{Float64,2} = softmax(x * net.w[1] + net.b[1])

# 損失関数: 交差エントロピー誤差
loss(net::Network, x::Array{Float64,2}, t::Array{Float64,2})::Float64 = cross_entropy_error(predict(net, x), t)

x = [0.6 0.9]
t = [0.0 0.0 1.0]

net = SimpleNet()

# 勾配計算
## 2 x 3 Array{Float64,2} [0.2 0.2 -0.4; 0.3 0.3 -0.6] になればOK
dW = numeric_gradient(w->loss(net, x, t), net.w[1])

2×3 Array{Float64,2}:
 0.2  0.2  -0.4
 0.3  0.3  -0.6